In [3]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [15]:
import json
import pprint
from collections import Counter
import string
import re

## Принты
Сделал два разных вывода, _nice_print_ для топ-списка, _good_print_ для таблички.

In [36]:
def nice_print(name, list_of_items, t=''):
    print(name)
    for item in list_of_items:
        print(list_of_items.index(item) + 1, item[0], t, item[1])


def good_print(col_1, col_2, ljust, list_of_items):
    print(col_1.ljust(ljust) + '\t' + col_2)
    print('-' * ljust * 2)
    for item in list_of_items:
        print(item[0].ljust(ljust) + '\t' + str(item[1]))

## Задание 1
Открываю файл и считываю его построчно, потом смотрю на длину получившегося списка

In [10]:
twits = []
for twit in open('hw_3_twitter.json'):
    twits.append(json.loads(twit))
print('всего %s твитов' % len(twits))

всего 2556 твитов


## Задание 2
Завожу счётчик и считаю, сколько твитов с пометкой __delete__, потом считаю проценты (с округлением до 2 знаков)

In [11]:
k = 0
for twit in twits:
    if 'delete' in twit:
        k += 1
print('удалённых твитов %s процентов от общего числа'
      % round(k / len(twits) * 100, 2))

удалённых твитов 14.16 процентов от общего числа


## Задание 3
Проверяю, есть ли в твите __lang__, если есть, то добавляю его значение в список. Потом прохожусь **Counter**-ом, выбираю 15 самых популярных и распечатываю.

In [28]:
langs = []
for twit in twits:
    if 'lang' in twit:
        langs.append(twit['lang'])
mc = Counter(langs).most_common(15)
nice_print('топ 15 самых популярных языков', mc, t='\t')

топ 15 самых популярных языков
1 en 	 719
2 ja 	 438
3 es 	 173
4 ko 	 149
5 th 	 123
6 ar 	 119
7 und 	 117
8 in 	 71
9 pt 	 69
10 fr 	 35
11 tr 	 30
12 tl 	 29
13 hi 	 23
14 ru 	 15
15 fa 	 8


## Задание 4
Сначала нахожу все **user_id** всех твитов (в том числе удалённых). Потом использую **Counter().most_common()**, чтобы отсортировать по количеству твитов. Затем добавляю пользователей в новый список, пока не попадётся пользователь с одним твитом. Печатаю в таблицу.

In [60]:
users = []
for twit in twits:
    if 'delete' in twit:
        users.append(twit['delete']['status']['user_id_str'])
    else:
        users.append(twit['user']['id_str'])
users = Counter(users).most_common()
mult_users = []
for user in users:
    if user[1] > 1:
        mult_users.append(user)
    else:
        break
print('%s пользователей, у которых больше одного твита\n' % len(mult_users))
good_print('id пользователя', 'количество твитов', 19, mult_users)

66 пользователей, у которых больше одного твита

id пользователя    	количество твитов
--------------------------------------
790593289934209025 	11
2750500049         	10
590742037          	10
808526517147815936 	6
4137811932         	4
273413828          	4
3137027515         	4
168350800          	4
2390342004         	3
88401106           	3
1040919699721084928	3
613339621          	3
2871443750         	2
730365119805292544 	2
992084216350294016 	2
822238094652882948 	2
1409357251         	2
581282101          	2
995919322282250240 	2
2317193324         	2
978499715657445377 	2
871711626352832513 	2
2245928100         	2
993031040          	2
174334388          	2
804631744896868354 	2
3718013776         	2
290401936          	2
3067130479         	2
849417895109156869 	2
1020277823343390720	2
958056194366754816 	2
572311397          	2
769094648438648832 	2
1017442172495331328	2
702487896935104513 	2
3552991            	2
995683537197158401 	2
121016179          	2
9472883153753

## Задание 5
Проверяю есть ли поле **entities**; если есть -- достаю оттуда список хэштегов и из каждого достаю текст. Потом топ как в _Задании 3_.

In [35]:
hashtags = []
for twit in twits:
    if 'entities' in twit:
        for hashtag in twit['entities']['hashtags']:
            hashtags.append(hashtag['text'])
hashtags = Counter(hashtags).most_common(20)
nice_print('топ 20 хэштегов', hashtags)

топ 20 хэштегов
1 BTS  17
2 방탄소년단  13
3 AMAs  11
4 人気投票ガチャ  8
5 태형  7
6 뷔  6
7 BTSinChicago  5
8 BTSLoveYourselfTour  5
9 오늘의방탄  5
10 PledgeForSwachhBharat  5
11 MPN  5
12 PCAs  4
13 V  4
14 시카고1회차공연  4
15 เป๊กผลิตโชค  4
16 JIMIN  4
17 running  3
18 NCT  3
19 지민  3
20 WajahmuPlastik  3


## Задание 6
Сначала собираю все слова в англоязычных твитах, которые не ретвиты (нет **retweeted_status** и **quoted_status**). Потом чищу все слова от пунктуации с помощью списка, полученного из **string.punctuation** и дополненного мной **'…'**. Все чистенькие слова добавляю в новый список и прохожусь **Counter**-ом, потом печатаю, как в одном из предыдущих заданий.

In [58]:
raw_words = []
punc = string.punctuation + '…'
punc = punc.replace("'", "")
for twit in twits:
    if 'lang' in twit:
        if twit['lang'] == 'en':
            if 'retweeted_status' not in twit and 'quoted_status' not in twit:
                raw_words.extend(twit['text'].split())
words = []
for rword in raw_words:
    if rword.endswith("'"):
        rword = rword[:-1]
    if rword.startswith("'"):
        rword = rword[1:]
    for item in punc:
        if item in rword:
            rword = rword.replace(item, '')
    if rword != '':
        words.append(rword.lower())
freq_d = Counter(words)
for_print = freq_d.most_common(40)
good_print('слово', 'количество упоминаний', 31, for_print)

слово                          	количество упоминаний
--------------------------------------------------------------
the                            	107
to                             	79
a                              	67
i                              	60
and                            	58
you                            	45
is                             	41
for                            	40
of                             	40
it                             	38
in                             	32
that                           	30
my                             	26
on                             	25
me                             	25
be                             	22
this                           	20
are                            	20
have                           	19
so                             	18
but                            	17
at                             	17
your                           	16
get                            	16
not                            	16
more   

## Задание 7
Собираю данные обо всех подписчиках всех пользователей, потом уаляю дубли с помощью **set**. Дальше сортирую список кортежей по второму элементу и беру первые 10, печатаю.

In [41]:
fols = []
for twit in twits:
    if 'user' in twit:
        if 'followers_count' in twit['user']:
            fols.append((
                twit['user']['screen_name'],
                twit['user']['followers_count']
            ))
fols = set(fols)
fols = sorted(fols, key=lambda x: x[1], reverse=True)
fols = fols[:10]
nice_print('топ 10 самых популярных авторов:', fols)

топ 10 самых популярных авторов:
1 FIL0S0FIA  2521403
2 FitnessMagazine  1491309
3 malaysiakini  1206759
4 NYTScience  1137374
5 GramaticaReal  625463
6 tgrthabertv  392472
7 TheSunFootball  383698
8 Melbourne  374222
9 Roznama_Express  318189
10 burger_boogie  311319


## Задание 8
Завожу пустой список **sources** и сразу задаю паттерн для поиска источника. В каждом твите, если источник указан, кладу его в список, потом делаю частотный словарь и выбираю оттуда 10 первых элементов. Печатаю.

In [42]:
sources = []
pattern = r'<a[^>]*>(.*)</a>'
for twit in twits:
    if 'source' in twit:
        source = re.search(pattern, twit['source']).groups()
        sources.extend(source)
scs = Counter(sources).most_common(10)
nice_print('топ 10 источников твитов:', scs)

топ 10 источников твитов:
1 Twitter for iPhone  800
2 Twitter for Android  695
3 Twitter Web Client  140
4 twittbot.net  122
5 Twitter Lite  51
6 Twitter for iPad  28
7 TweetDeck  23
8 Facebook  17
9 IFTTT  14
10 تطبيق قرآني  10
